# 📌 IMPORTS

In [1]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, SMAIndicator, MACD
from ta.volatility import BollingerBands
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import accuracy_score


# 📌Stock list

In [2]:
stocks = [
    "RELIANCE", "TCS", "INFY", "HDFCBANK", "ICICIBANK",
    "LT", "BAJFINANCE", "SBIN", "HINDUNILVR", "MARUTI"
]

# 📌 Function to compute indicators and train model


In [4]:
for stock in stocks:
    print(f"📈 Processing {stock}...")

    df = pd.read_csv(f"data/{stock}.csv", parse_dates=["Date"])
    df.set_index("Date", inplace=True)
    # print(df.head())
    # print("___________________________________________________________________-")
    df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
    # print(df.head())
    df.dropna(subset=["Close"], inplace=True)
    # print("___________________________________________________________________-")

    # print(df.head())
    

    try:
        df["SMA_14"] = SMAIndicator(df["Close"], window=14).sma_indicator()
        df["EMA_14"] = EMAIndicator(df["Close"], window=14).ema_indicator()
        df["RSI"] = RSIIndicator(df["Close"], window=14).rsi()

        macd = MACD(df["Close"])
        df["MACD"] = macd.macd()
        df["MACD_signal"] = macd.macd_signal()

        bb = BollingerBands(df["Close"])
        df["BB_upper"] = bb.bollinger_hband()
        df["BB_lower"] = bb.bollinger_lband()

        # Target: 1 if next day price goes UP
        df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
        df.dropna(inplace=True)

        # Features
        features = ["SMA_14", "EMA_14", "RSI", "MACD", "MACD_signal", "BB_upper", "BB_lower"]
        X = df[features]
        y = df["Target"]

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        model = RandomForestClassifier(random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"✅ {stock} model trained with accuracy: {accuracy*100:.2f}\n")


        # Save model and scaler
        joblib.dump(model, f"models/{stock}.pkl")
        joblib.dump(scaler, f"models/{stock}_scaler.pkl")

        df.to_csv(f"data/{stock}_features.csv")

        print(f"✅ {stock} model & scaler saved.\n")

    except Exception as e:
        print(f"❌ Error processing {stock}: {e}\n")

📈 Processing RELIANCE...
✅ RELIANCE model trained with accuracy: 50.00

✅ RELIANCE model & scaler saved.

📈 Processing TCS...
✅ TCS model trained with accuracy: 56.42

✅ TCS model & scaler saved.

📈 Processing INFY...
✅ INFY model trained with accuracy: 53.50

✅ INFY model & scaler saved.

📈 Processing HDFCBANK...
✅ HDFCBANK model trained with accuracy: 50.39

✅ HDFCBANK model & scaler saved.

📈 Processing ICICIBANK...
✅ ICICIBANK model trained with accuracy: 51.95

✅ ICICIBANK model & scaler saved.

📈 Processing LT...
✅ LT model trained with accuracy: 51.56

✅ LT model & scaler saved.

📈 Processing BAJFINANCE...
✅ BAJFINANCE model trained with accuracy: 50.78

✅ BAJFINANCE model & scaler saved.

📈 Processing SBIN...
✅ SBIN model trained with accuracy: 53.70

✅ SBIN model & scaler saved.

📈 Processing HINDUNILVR...
✅ HINDUNILVR model trained with accuracy: 53.50

✅ HINDUNILVR model & scaler saved.

📈 Processing MARUTI...
✅ MARUTI model trained with accuracy: 50.78

✅ MARUTI model & sca